In [ ]:
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

import pickle
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import random
from uncertainties import ufloat
from scipy.optimize import curve_fit 
import scipy.stats as stats 

from ROOT import TLorentzVector
from ROOT import TVector3

import warnings
warnings.filterwarnings("ignore")

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *
from SystematicsHelpers import *

In [ ]:
c_cm_per_ns = 29.9792458

SBND_AV = [{
      "xmin": -200,
      "xmax": 0,
      "ymin": -200,
      "ymax": 200,  
      "zmin": 0,
      "zmax": 500,
    },
    {
      "xmin": 0,
      "xmax": 200,
      "ymin": -200,
      "ymax": 200,
      "zmin": 0,      
      "zmax": 500,
    }
]

#define binning
bins = np.arange(0, 20, 1)
bins_mid = np.convolve(bins, [0.5, 0.5], "valid")

print(bins)
print(bins_mid)

#buck spacing
bucket_dt = 18.936

#shift in truth to pds reco 
reco_shift = 1.2 #1.46

#smear
smear_sigma = 0.00093

step = 0.2
predict_x = np.arange(0, 19+step, step)

<h1> Read MCTruth Neutrino Containing Dirt + TPC Neutrinos </h1>

In [ ]:
def prep_nudf(df):
    #APPLY AV cut: only keep vtx inside TPc
    df= df.loc[(df['nu_vtx_x'] > SBND_AV[0]['xmin'])
             & (df['nu_vtx_x'] < SBND_AV[1]['xmax'])
             & (df['nu_vtx_y'] > SBND_AV[0]['ymin'])
             & (df['nu_vtx_y'] < SBND_AV[1]['ymax'])
             & (df['nu_vtx_z'] > SBND_AV[0]['zmin'])
             & (df['nu_vtx_z'] < SBND_AV[1]['zmax'])]

    return df

def compute_nuZ(df):
    #compute arrival time
    df['nu_enterT'] = df['nu_vtx_t'] - df['nu_vtx_z'] / c_cm_per_ns
    
    #add shift due to reco
    df['nu_enterT_shift'] = df['nu_enterT'] + reco_shift
        
    # mean and standard deviation
    mu, sigma = 1, smear_sigma #--> this value smear sigma to ~1.73ns

    n_univ = len(df['nu_enterT'])
    random_arr = np.random.normal(mu, sigma, n_univ)
    
    df['nu_enterT_smear'] = dfNU['nu_enterT_shift'] * random_arr

    #take modulus
    df['mod_t_smear'] = df['nu_enterT_smear'] % bucket_dt
    df['mod_t'] = df['nu_enterT'] % bucket_dt
    
    return df

def add_mean_shift(para, shift):
    return [para[0], para[1]+shift, para[2], para[3]]

In [ ]:
nu_path = "../pkl_files/v3_May2024/df_mctruth_v3_nu.pkl"
file = open(nu_path, 'rb')
dfNU = pickle.load(file)
file.close()

dfNU = prep_nudf(dfNU)
dfNU = compute_nuZ(dfNU)

<h1>Gaussian Fit MCTruth w/ Normalisation to 1</h1>

In [ ]:
#Normal Gaussian 
def gaus(X, A, mean, sigma, C):
    return A * np.exp( -( X - mean )**2 / ( 2 * sigma**2 ) ) + C

#Normal Gaussian without constant, should be no constant since we don't consider intime cosmic at the moment
def gaus_noC(X, A, mean, sigma):
    return A * np.exp( -( X - mean )**2 / ( 2 * sigma**2 ) )

def fit_cv(hist):

    prediction = [max(hist), 10, 1.308]

    lower_bound = [-np.inf, -np.inf, -np.inf]

    upper_bound = [np.inf, np.inf, np.inf]

    param_bounds=(lower_bound, upper_bound)

    parameters, covariance = curve_fit(gaus_noC, bins_mid, hist, p0 = prediction, bounds=param_bounds) 

    sigma_parameters = np.sqrt(np.diag(covariance))
    
    return parameters, sigma_parameters

def fit_legend(hist, predict, parameters, sigma_parameters):
    #error
    A = ufloat(parameters[0], sigma_parameters[0])
    mean= ufloat(parameters[1], sigma_parameters[1])
    sigma = ufloat(parameters[2], sigma_parameters[2])

    legend = "A = {}\nmean = {}\nsigma = {}".format(A, mean, sigma)
    
    chi2 = np.sum((hist - predict)**2 / predict)
    chi2_txt = "\nchi2 = {0:.4f}".format(chi2)

    legend = legend + chi2_txt
    
    return legend

def make_pretty_fit_legend(parameters, sigma_parameters):

    legend = "Gaussian Fit\n"
    legend = legend + r"$\mu$"+" = {0:.2f}".format(parameters[1]) + r"$\pm$" +"{0:.3f}\n".format(sigma_parameters[1])
    legend = legend + r"$\sigma$"+" = {0:.2f}".format(parameters[2]) + r"$\pm$" +"{0:.3f}".format(sigma_parameters[2])

    return legend

In [ ]:
cv,_ = np.histogram(dfNU['mod_t'], bins = bins, density = True)

cv_smear,_ = np.histogram(dfNU['mod_t_smear'], bins = bins, density = True)

#=====================================================================#

paraCV, sigCV = fit_cv(cv)
paraCVsmear, sigCVsmear = fit_cv(cv_smear)

#=====================================================================#
#predictCV = gaus_noC(bins_mid, *paraCV)
#predictCVsmear = gaus_noC(bins_mid, *paraCVsmear)

predictCV = gaus_noC(predict_x, *paraCV)
predictCVsmear = gaus_noC(predict_x, *paraCVsmear)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), sharey = False)

#-----------------------------------------------------------------#

ax1.step(bins, np.insert(cv, 0,0)
         , color = col_dict["Spearmint"]
         , label =  "Truth"
)
ax1.plot(predict_x, predictCV
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCV, sigCV)
        )
#-----------------------------------------------------------------#

ax2.step(bins, np.insert(cv_smear, 0,0)
         , color = col_dict["Peach"]
         , label =  "Truth Smeared"
)

ax2.plot(predict_x, predictCVsmear
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCVsmear, sigCVsmear)
        )
#-----------------------------------------------------------------#

plot_tick(ax1, 16)
plot_title(ax1, "", 'Arrival Time [ns]',  r"Counts [a.u.]", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 0.4)
ax1.legend(loc = 'upper right',fontsize = 12, ncol = 1)

#-----------------------------------------------------------------#
plot_tick(ax2, 16)
plot_title(ax2, "", 'Arrival Time [ns]',  r"Counts [a.u.]", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 0.4)

ax2.legend(loc = 'upper right',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig("./truth_smear_gaus.png", dpi = 200)

plt.show()

<h1>Scale Up Please - Change to Your Own Reco Background File Here</h1>

In [ ]:
#Loading background histogram after full selection to get the final counts of background events 
#Bkg = Neutrino + Cosmics scaled to POT
bkg_dict = np.load("../pkl_files/v3_April2024/bkg_v3_dict_truly_final_loosy.npy",allow_pickle='TRUE').item()
#bkg_dict = np.load("../pkl_files/v3_April2024/bkg_v3_dict_truly_final.npy",allow_pickle='TRUE').item()

bkg_array = np.array(bkg_dict['bkg'])
total_bkg_events = np.sum(bkg_array)

print("Total bkg events = {0:.0f}".format(total_bkg_events))

In [ ]:
#Number of Background Events = scale parameter of the Gaussian
scale = total_bkg_events

In [ ]:
def scale_parameter(para, scale):
    scale_para = [para[0]*scale, para[1], para[2]]
    scale_para = np.array(scale_para)
    return scale_para
    
paraCV_scale = scale_parameter(paraCV, scale)
paraCVsmear_scale = scale_parameter(paraCVsmear, scale)
sigCV_scale = scale_parameter(sigCV, scale)
sigCVsmear_scale = scale_parameter(sigCVsmear, scale)

In [ ]:
#fit a Gaussian to the reco distribution for sanity check
paraCV_reco, sigCV_reco = fit_cv(bkg_array)
predictCV_reco = gaus_noC(bins_mid, *paraCV_reco)

#use prediction from truth
predictCV_scale = gaus_noC(bins_mid, *paraCV_scale)
predictCVsmear_scale = gaus_noC(bins_mid, *paraCVsmear_scale)

#plot confidence interval
predictCV_reco_ub = gaus_noC(bins_mid, *(paraCV_reco + sigCV_reco))
predictCV_scale_ub = gaus_noC(bins_mid, *(paraCV_scale + sigCV_scale))
predictCVsmear_scale_ub = gaus_noC(bins_mid, *(paraCVsmear_scale + sigCVsmear_scale))

predictCV_reco_lb = gaus_noC(bins_mid, *(paraCV_reco - sigCV_reco))
predictCV_scale_lb = gaus_noC(bins_mid,  *(paraCV_scale - sigCV_scale))
predictCVsmear_scale_lb = gaus_noC(bins_mid, *(paraCVsmear_scale - sigCVsmear_scale))

In [ ]:
#=====================================================================#
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (16, 4), sharey = True, sharex =True)

#plot Reco
ax1.step(bins, np.insert(bkg_array, 0,0), color = 'blue', label = 'Reco')
ax2.step(bins, np.insert(bkg_array, 0,0), color = 'blue', label = 'Reco')
ax3.step(bins, np.insert(bkg_array, 0,0), color = 'blue', label = 'Reco')

#plot Gaussian fit
ax1.plot(bins_mid, predictCV_reco, 'pink', label = fit_legend(bkg_array, predictCV_reco, paraCV_reco, sigCV_reco))
ax2.plot(bins_mid, predictCV_scale, 'green', label = fit_legend(bkg_array, predictCV_scale, paraCV_scale, sigCV_scale))
ax3.plot(bins_mid, predictCVsmear_scale, 'red', label = fit_legend(bkg_array, predictCVsmear_scale, paraCVsmear_scale, sigCVsmear_scale))

#plot error bounds
ax1.fill_between(bins_mid, predictCV_reco_lb, predictCV_reco_ub, color ='grey', alpha= 0.2)
ax2.fill_between(bins_mid, predictCV_scale_ub, predictCV_scale_lb, color ='grey', alpha= 0.2)
ax3.fill_between(bins_mid, predictCVsmear_scale_ub, predictCVsmear_scale_lb, color ='grey', alpha= 0.2)

ax1.legend(loc='upper right', ncol = 3)
ax2.legend(loc='upper right', ncol = 3)
ax3.legend(loc='upper right', ncol = 3)

ax1.set_title("Reco + Gaussian Fit To Reco")
ax2.set_title("Reco + Gaussian Fit To MCTruth")
ax3.set_title("Reco + Gaussian Fit To Smeared MCTruth")


#ax1.set_ylim(0, 300)
#ax1.set_xlim(0, 5)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15,4), sharey = False)

#-----------------------------------------------------------------#

ax1.step(bins, np.insert(cv*scale, 0,0)
         , color = col_dict["Spearmint"]
         , label =  r"Truth"
)

predictCV_scale = gaus_noC(predict_x, *paraCV_scale)
ax1.plot(predict_x, predictCV_scale
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCV_scale, sigCV_scale)
        )
#-----------------------------------------------------------------#

ax2.step(bins, np.insert(cv_smear*scale, 0,0)
         , color = col_dict["Peach"]
         , label =  r"Truth Smeared"
)
predictCVsmear_scale = gaus_noC(predict_x, *paraCVsmear_scale)
ax2.plot(predict_x, predictCVsmear_scale
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCVsmear_scale, sigCVsmear_scale)
        )
#-----------------------------------------------------------------#

ax3.step(bins, np.insert(bkg_array, 0,0)
         , color = col_dict["Teal"]
         , label =  r"Reco + $\nu\pi^0$ Selection"
)

predictCV_reco = gaus_noC(predict_x, *paraCV_reco)
ax3.plot(predict_x, predictCV_reco
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCV_reco, sigCV_reco)
        )
#-----------------------------------------------------------------#

plot_tick(ax1, 16)
plot_title(ax1, "", 'Arrival Time [ns]',  r"Counts", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 4000)
ax1.legend(loc = 'upper right',fontsize = 12, ncol = 1)

#-----------------------------------------------------------------#
plot_tick(ax2, 16)
plot_title(ax2, "", 'Arrival Time [ns]',  r"Counts", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 4000)

ax2.legend(loc = 'upper right',fontsize = 12, ncol = 1)

#-----------------------------------------------------------------#

plot_tick(ax3, 16)
plot_title(ax3, "", 'Arrival Time [ns]',  r"Counts", 16)

ax3.set_xlim(0, 19)
ax3.set_ylim(0, 4000)
ax3.legend(loc = 'upper right',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig("./truth_smear_reco_gaus.png", dpi = 200)

plt.show()

<h1>Neutrino + Cosmics Bkg After Cosmics Rejection Scaled To POT</h1>

In [ ]:
nu_path = "../pkl_files/v3_April2024/df_postcosmics_v3_nu.pkl"
file = open(nu_path, 'rb')
dfNUreco = pickle.load(file)
file.close()

dfNUreco = dfNUreco[dfNUreco['slc_true_event_type'] != 9]

In [ ]:
cv_reco,_ = np.histogram(dfNUreco['mod_t'], bins = bins, density = True)

paraReco, sigReco = fit_cv(cv_reco)

In [ ]:
predictTruth = gaus_noC(predict_x, *paraCV)

predictReco = gaus_noC(predict_x, *paraReco)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), sharey = False)

#-----------------------------------------------------------------#

ax1.step(bins, np.insert(cv, 0,0)
         , color = col_dict["Spearmint"]
         , label =  "Truth"
)
ax1.plot(predict_x, predictTruth
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraCV, sigCV)
        )
#-----------------------------------------------------------------#

ax2.step(bins, np.insert(cv_reco, 0,0)
         , color = col_dict["Teal"]
         , label =  "Reco"
)

ax2.plot(predict_x, predictReco
         , c = col_dict["Coral"]
         , label = make_pretty_fit_legend(paraReco, sigReco)
        )
#-----------------------------------------------------------------#

plot_tick(ax1, 16)
plot_title(ax1, "", 'Arrival Time [ns]',  r"Counts [a.u.]", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 0.4)
ax1.legend(loc = 'upper right',fontsize = 12, ncol = 1)

#-----------------------------------------------------------------#
plot_tick(ax2, 16)
plot_title(ax2, "", 'Arrival Time [ns]',  r"Counts [a.u.]", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 0.4)

ax2.legend(loc = 'upper right',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
fig.tight_layout()

#plt.savefig("./truth_reco_gaus.png", dpi = 200)

plt.show()

<h1> Read in MeVPrtlTruth </h1>

In [ ]:
def read_hnlroot2df(fname):
    #SPECIFY BRANCHES
    branches = [
    "mevprtl_mom"
    , "decay_pos"
    , "pot"
    , "flux_weight"
    , "ray_weight"
    , "decay_weight"
    ]
    
    folder = "generator"
    tname = "mevprtl_gen"
    tree = uproot.open(fname)[folder][tname]
    df = tree.arrays(branches, library="pd")
    return df

def prep_hnldf(df):
    #FLATTEN MULTI INDEX COLUMN FOR EASY HANDLING
    df.columns = ['_'.join(col) for col in df.columns.values]

    #RENAME COLUMN FOR CLARITY
    df = df.rename(columns = {
    "mevprtl_mom_fP_fX" : "hnl_mom_pX"
    , "mevprtl_mom_fP_fY" : "hnl_mom_pY"
    , "mevprtl_mom_fP_fZ" : "hnl_mom_pZ"
    , "mevprtl_mom_fE_" : "hnl_mom_E"
    , "decay_pos_fP_fX" : "hnl_decay_X"
    , "decay_pos_fP_fY" : "hnl_decay_Y"
    , "decay_pos_fP_fZ" : "hnl_decay_Z"
    , "decay_pos_fE_" : "hnl_decay_t"
    , "pot__" : "pot"
    , "flux_weight__" : "fluxw"
    , "ray_weight__" : "rayw"
    , "decay_weight__" : "decayw"
    }
    )
    
    #APPLY AV cut the HNL DECAY VERTEX
    df= df.loc[(df['hnl_decay_X'] > SBND_AV[0]['xmin'])
             & (df['hnl_decay_X'] < SBND_AV[1]['xmax'])
             & (df['hnl_decay_Y'] > SBND_AV[0]['ymin'])
             & (df['hnl_decay_Y'] < SBND_AV[1]['ymax'])
             & (df['hnl_decay_Z'] > SBND_AV[0]['zmin'])
             & (df['hnl_decay_Z'] < SBND_AV[1]['zmax'])]
    
    #SIMULATED POT: the dataframe contains 2 rows per HNL for daughter, so don't wannt duplicate for counting POT
    df_no_dupes = df.drop_duplicates(subset=["hnl_mom_pX"])
    simpot = df_no_dupes['pot'].sum() 

    #predicted POT for SBND
    totalpot = 10**21

    #Deweight
    df['dew'] = df['fluxw']*df['rayw']*df['decayw'] / simpot * totalpot

    return df

def correct_Z(df):
     
    hnlpX = df['hnl_mom_pX']
    hnlpY = df['hnl_mom_pY']
    hnlpZ = df['hnl_mom_pZ']
    hnlE = df['hnl_mom_E']

    hnldposZ = df['hnl_decay_Z']
    hnldposT = df['hnl_decay_t']
    
    hnlcorr = []
    
    for (X, Y, Z, E, z, t) in zip(hnlpX, hnlpY, hnlpZ, hnlE, hnldposZ, hnldposT):
        
        vec = TLorentzVector (X, Y, Z, E)
        vel = vec.Beta() * c_cm_per_ns
        
        tofcorr = z / vel
        
        hnlcorr.append(tofcorr)
     
    #compute arrival time
    df['hnl_enter_t'] = df['hnl_decay_t'] - hnlcorr
    
    #add shift due to reco
    df['hnl_enter_t_shift'] = df['hnl_enter_t'] + reco_shift
    
    # mean and standard deviation
    mu, sigma = 1, smear_sigma #--> this value smear sigma to ~1.73ns

    n_univ = len(df['hnl_enter_t_shift'])
    random_arr = np.random.normal(mu, sigma, n_univ)
    df['hnl_enter_t_smear'] = df['hnl_enter_t_shift'] * random_arr
    
    #take modulus
    df['mod_t'] = df['hnl_enter_t'] % bucket_dt
    df['mod_t_smear'] = df['hnl_enter_t_smear'] % bucket_dt
    
    return df

def trim(df):
    df = df.reset_index()
    
    #keep relevant columns: time and weight
    df = df[['dew', 'mod_t', 'mod_t_smear']]
    
    #drop duplicates: due to having 2 daughters 
    df = df.drop_duplicates(subset=["dew","mod_t"])
    
    return df

def scale_U(df, m):
    scale = (scale_U_list[m]/sim_U_list[m])**2
    df['dew'] = df['dew'] * scale
    return df

<h1> Change to Your Own MeVPrtl Truth Files and Mass List Here </h1>

In [ ]:
#UPDATE THIS BLOCK TO YOUR OWN PARAMETERS!

mass_list = [140, 160, 180, 200, 220, 240, 260]

col_list = {
    140: col_dict['Lavender']
    ,160: col_dict['Purple']
    ,180: col_dict['Teal']
    ,200: col_dict['Aqua']
    ,220: col_dict['Spearmint']
    ,240: col_dict['Peach']
    ,260: col_dict['Coral']
    
}

sim_U_list = { #simulated U 
    140: 1e-05
    , 160: 1e-06
    , 180: 1e-06
    , 200: 1e-07
    , 220: 1e-07
    , 240: 1e-07
    , 260: 1e-07
}

scale_U_list = { #scale to U that gives the same event rate, this is the same U for reco
    140: 2.1379236405595657e-06
    , 160: 3.5971094624542994e-07
    , 180: 1.6863609063111095e-07
    , 200: 1e-07
    , 220: 6.57368812318177e-08
    , 240: 4.6359838200981854e-08
    , 260: 3.459579081079871e-08
}

plot_U_list = { #scale to U that gives the same event rate, this is the same U for reco
    140: 6.76e-06
    , 160: 1.14e-06
    , 180: 5.33e-07
    , 200: 3.16e-07
    , 220: 2.08e-07
    , 240: 1.47e-07
    , 260: 1.091e-07
}

In [ ]:
hnl_df_dict = {}

for m in mass_list:
    hnl_path = "../root_files/nupi0_truth_m{}.root".format(m)
    df = read_hnlroot2df(hnl_path) #read in ntuple
    df = prep_hnldf(df) #change some name, compute deweight values
    df = correct_Z(df) #compute arrival time
    
    df = trim(df) #trim the dataframe to easier
    df = scale_U(df, m) #scale simulated U to another U. Comment this out if you don't want it.
    
    hnl_df_dict[m] = df

In [ ]:
hnl_truth_dict = {}
hnl_smear_dict = {}

for m in mass_list:
    hist,_ = np.histogram(hnl_df_dict[m]['mod_t'], bins = bins, weights = hnl_df_dict[m]['dew'])
    
    hnl_truth_dict[m] = hist
    
    hist,_ = np.histogram(hnl_df_dict[m]['mod_t_smear'], bins = bins, weights = hnl_df_dict[m]['dew'])
    
    hnl_smear_dict[m] = hist

<h1> Read in HNL Reco - Change to your own file here </h1>

In [ ]:
#Loading HNL histogram after full selection for truth vs reco comparison at the same U 
hnl_reco_dict = {}

for m in mass_list:

    hnl_dict = np.load("../pkl_files/v3_April2024/hnl_m{}_v3_dict_truly_final_loosy.npy".format(m),allow_pickle='TRUE').item()
    hnl_reco_dict[m] = np.array(hnl_dict['signal'])

<h1> Compare Reco To Truth To See if We Should Apply Some Efficiency ? </h1>

In [ ]:
for m in mass_list:
    fig, ax = plt.subplots(1,1, figsize = (6, 4), sharey = True)
    ax.step(bins, np.insert(hnl_truth_dict[m],0,0), color = col_list[m], label = 'Truth')
    ax.step(bins, np.insert(hnl_smear_dict[m],0,0), color = "black", label = 'Truth Smeared')
    ax.step(bins, np.insert(hnl_reco_dict[m], 0,0), color ='grey', label = 'Reco')
    ax.legend()
    ax.set_title(m)
    plt.show()

<h1> I Ballpark Some Efficiency For Truth Here To Account For Reco/Selection - Change to Your Own</h1>

In [ ]:
#This is what I do for my channel but you can come up for your own

#Truly Final Loosy Reco Selection Efficiency
#M = 200, U = 1e-07, start true counts = 1481 events
#After full selection: counts = 414 --> Eff = 414 / 1481 = 28%
#After first + last 4bins: counts = 142 --> Eff = 142 / 1481 = 10%

#Gonna pick a flat efficiency for reco that gives me roughly event rate as reco

#Sanity check: total truth --> same mass and coupling for truth

total = np.sum(hnl_smear_dict[200])
print("total events = " + str(total))

#Apply a flat efficiency of 30% --> Let's be optimistic
efficiency = 0.3
select = hnl_smear_dict[200] * efficiency
print("total events after flat eff = " + str(np.sum(select)))

#Keep only the first and last 4 bins
select = np.sum(select[0:4]) + np.sum(select[-4:])
print("total events after first/last 4 bins  = " + str(np.sum(select)))

In [ ]:
#Sanity check comparing reco and truth * mock efficiency. 
#Might not be very good agreement since reco selection is very differenct from applying a flat effciency

for m in mass_list:
    fig, ax = plt.subplots(1,1, figsize = (6, 4), sharey = True)
    #apply efficiency here
    ax.step(bins, np.insert(hnl_truth_dict[m]* efficiency,0,0), color = col_list[m], label = 'Truth')
    ax.step(bins, np.insert(hnl_smear_dict[m]* efficiency,0,0), color = "black", label = 'Truth Smeared')
    ax.step(bins, np.insert(hnl_reco_dict[m], 0,0), color ='grey', label = 'Reco')
    ax.legend()
    ax.set_title(m)
    plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15,4), sharey = False)
scaleHNL = 10
#-----------------------------------------------------------------#

hnllabel = r'200 MeV HNL $\nu\pi^0$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(plot_U_list[200],2,2))

ax1.step(bins, np.insert(hnl_truth_dict[200] * efficiency * scaleHNL,0,0)
         , color = col_dict['Spearmint']
         , label =  hnllabel+ '\n' + r"Truth + 30% Efficiency"
)

ax2.step(bins, np.insert(hnl_smear_dict[200] * efficiency * scaleHNL,0,0)
         , color = col_dict['Peach']
         , label =   hnllabel+ '\n' +r"Truth Smeared + 30% Efficiency"
)

ax3.step(bins, np.insert(hnl_reco_dict[200] * scaleHNL,0,0)
         , color = col_dict['Teal']
         , label =   hnllabel+ '\n' +r"Reco + $\nu\pi^0$ Selection"
)
#-----------------------------------------------------------------#

plot_tick(ax1, 16)
plot_title(ax1, "", 'Arrival Time [ns]',  r"Counts", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 700)
ax1.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#

plot_tick(ax2, 16)
plot_title(ax2, "", 'Arrival Time [ns]',  r"Counts", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 700)
ax2.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#

plot_tick(ax3, 16)
plot_title(ax3, "", 'Arrival Time [ns]',  r"Counts", 16)

ax3.set_xlim(0, 19)
ax3.set_ylim(0, 700)
ax3.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig("./truth_smear_reco_gaus_HNL.png", dpi = 200)

plt.show()

<h1>Overlay with Background MeVTruth HNL</h1>

In [ ]:
predictCV_reco = gaus_noC(bins_mid, *paraCV_reco)
predictCV_scale = gaus_noC(bins_mid, *paraCV_scale)
predictCVsmear_scale = gaus_noC(bins_mid, *paraCVsmear_scale)

In [ ]:
#=====================================================================#
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (18, 4), sharey = True)

#plot Bkg Reco
ax1.step(bins, np.insert(bkg_array, 0,0), color = 'grey')
ax2.step(bins, np.insert(predictCV_reco, 0,0), color = 'grey')
ax3.step(bins, np.insert(predictCV_scale, 0,0), color = 'grey')
ax4.step(bins, np.insert(predictCVsmear_scale, 0,0), color = 'grey')

#plot HNL truth fit
for m in mass_list:
    ax1.step(bins, np.insert(hnl_smear_dict[m]* efficiency,0,0), color = col_list[m])
    ax2.step(bins, np.insert(hnl_smear_dict[m]* efficiency,0,0), color = col_list[m])
    ax3.step(bins, np.insert(hnl_smear_dict[m]* efficiency,0,0), color = col_list[m])
    ax4.step(bins, np.insert(hnl_smear_dict[m]* efficiency,0,0), color = col_list[m])

ax1.set_title("Reco Bkg")
ax2.set_title("Gaussian Fit To Reco Bkg\n sigma = {0:.2f} ns".format(paraCV_reco[2]))
ax3.set_title("Gaussian Fit To MCTruth\n sigma = {0:.2f} ns".format(paraCV_scale[2]))
ax4.set_title("Gaussian Fit To Smeared MCTruth\n sigma = {0:.2f} ns".format(paraCVsmear_scale[2]))

ax1.set_ylim(0, 80)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (20.5,4), sharey = False)

ax1.step(bins, np.insert(cv*scale, 0,0)
         , color = "grey"
         , label = r"SM $\nu$"
)

ax2.step(bins, np.insert(cv_smear*scale, 0,0)
         , color = "grey"
)

ax3.step(bins, np.insert(bkg_array, 0,0)
         , color = "grey"
)

#-----------------------------------------------------------------#
fitU = 3.16e-7

scaleHNL = 10

for m in mass_list:
    hnllabel = str(m) + r' MeV HNL $\nu\pi^0$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(plot_U_list[m],2,2))
    ax1.step(bins, np.insert(hnl_truth_dict[m] * efficiency * scaleHNL,0,0)
         , color = col_list[m]
         , label = hnllabel
    )

for m in mass_list:
    ax2.step(bins, np.insert(hnl_smear_dict[m] * efficiency * scaleHNL,0,0)
         , color = col_list[m]
    )
    
for m in mass_list:
    ax3.step(bins, np.insert(hnl_reco_dict[m] *  scaleHNL,0,0)
         , color = col_list[m]
    )

#-----------------------------------------------------------------#
plot_tick(ax1, 16)
plot_title(ax1, "Truth", 'Arrival Time [ns]',  r"Counts", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 3000)
#ax1.legend(loc = 'upper left',fontsize = 12, ncol = 1)
lgd= ax1.legend(bbox_to_anchor=(2.75, -0.2, 0.0001, 0.0001), fontsize = 12, ncol = 4)
#-----------------------------------------------------------------#

plot_tick(ax2, 16)
plot_title(ax2, "Truth Smeared", 'Arrival Time [ns]',  r"Counts", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 3000)
#ax2.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#

plot_tick(ax3, 16)
plot_title(ax3, r"Reco + $\nu\pi^0$ Selection", 'Arrival Time [ns]',  r"Counts", 16)

ax3.set_xlim(0, 19)
ax3.set_ylim(0, 3000)
#ax3.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
#fig.tight_layout()

plt.savefig("./truth_smear_reco_gaus_HNL_all_masses.png", bbox_inches='tight', dpi=200)

plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (20.5,4), sharey = False)

ax1.step(bins, np.insert(cv*scale, 0,0)
         , color = "grey"
        , label = r"SM $\nu$"
)

ax2.step(bins, np.insert(cv_smear*scale, 0,0)
         , color = "grey"
)

ax3.step(bins, np.insert(bkg_array, 0,0)
         , color = "grey"
)

#-----------------------------------------------------------------#
fitU = 3.16e-7

scaleHNL = 10

for m in mass_list:
    hnllabel = str(m) + r' MeV HNL $\nu\pi^0$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(plot_U_list[m],2,2))
    ax1.step(bins, np.insert(hnl_truth_dict[m] * efficiency * scaleHNL,0,0)
         , color = col_list[m]
         , label = hnllabel
    )

for m in mass_list:
    ax2.step(bins, np.insert(hnl_smear_dict[m] * efficiency * scaleHNL,0,0)
         , color = col_list[m]
    )
    
for m in mass_list:
    ax3.step(bins, np.insert(hnl_reco_dict[m] *  scaleHNL,0,0)
         , color = col_list[m]
    )

#-----------------------------------------------------------------#
plot_tick(ax1, 16)
plot_title(ax1, "Truth", 'Arrival Time [ns]',  r"Counts", 16)

ax1.set_xlim(0, 19)
ax1.set_ylim(0, 550)
#ax1.legend(loc = 'upper left',fontsize = 12, ncol = 1)
lgd= ax1.legend(bbox_to_anchor=(2.75, -0.2, 0.0001, 0.0001), fontsize = 12, ncol = 4)
#-----------------------------------------------------------------#

plot_tick(ax2, 16)
plot_title(ax2, "Truth Smeared", 'Arrival Time [ns]',  r"Counts", 16)

ax2.set_xlim(0, 19)
ax2.set_ylim(0, 550)
#ax2.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#

plot_tick(ax3, 16)
plot_title(ax3, r"Reco + $\nu\pi^0$ Selection", 'Arrival Time [ns]',  r"Counts", 16)

ax3.set_xlim(0, 19)
ax3.set_ylim(0, 550)
#ax3.legend(loc = 'upper left',fontsize = 12, ncol = 1)
#-----------------------------------------------------------------#
#fig.tight_layout()

plt.savefig("./truth_smear_reco_gaus_HNL_all_masses_zoom.png", bbox_inches='tight', dpi=200)

plt.show()

<h1>Save Truth Background and Signal For Fitting in PYHF</h1>